In [15]:
import requests

##### The Odds API Setup
- 500 request limit on Free tier
- Important note: the lines are delayed by several minutes, meaning I may need another scraping tool

In [71]:
API_KEY = "fc35e20b0a8d983f77fc2fa9257a35c7"
SPORTS_URL = f"https://api.the-odds-api.com/v4/sports?apiKey={API_KEY}"
ODDS_URL = f"https://api.the-odds-api.com/v4/sports/basketball_nba/odds/?apiKey={API_KEY}&regions=us&markets=h2h,spreads&oddsFormat=american"


response = requests.get(ODDS_URL)
sportsbook_data = response.json()

In [86]:
nested_first = json_normalize(sportsbook_data, "bookmakers")
json_normalize(nested_first.to_dict(orient="records"))
json_normalize(sportsbook_data)

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
0,b2b80d981ff4426f188472bf545a5325,basketball_nba,NBA,2024-03-06T03:05:57Z,Denver Nuggets,Phoenix Suns,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
1,fdd137330111c44ff0e65e014723480d,basketball_nba,NBA,2024-03-07T00:10:00Z,Washington Wizards,Orlando Magic,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
2,c3dee32711c68f93c75549836115f334,basketball_nba,NBA,2024-03-07T00:40:00Z,Atlanta Hawks,Cleveland Cavaliers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
3,827bed6b078ff6ffad3e5cb5ff75c01e,basketball_nba,NBA,2024-03-07T00:40:00Z,Houston Rockets,Los Angeles Clippers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
4,939bc5ec699ebad1d13a780e8a339e7d,basketball_nba,NBA,2024-03-07T00:40:00Z,Philadelphia 76ers,Memphis Grizzlies,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
5,78b5b9c45d27d06e0be2ab258b2ec102,basketball_nba,NBA,2024-03-07T02:10:00Z,Utah Jazz,Chicago Bulls,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
6,ad573989eea644c71f5ba1585e8317d4,basketball_nba,NBA,2024-03-07T03:00:00Z,Golden State Warriors,Milwaukee Bucks,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
7,7f43dbe5080b44ae90836bbcbc659677,basketball_nba,NBA,2024-03-07T03:10:00Z,Portland Trail Blazers,Oklahoma City Thunder,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
8,589eed81cc5543b6147d9b6b1c973c08,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings,"[{'key': 'draftkings', 'title': 'DraftKings', ..."


In [87]:
sportsbook_data

[{'id': 'b2b80d981ff4426f188472bf545a5325',
  'sport_key': 'basketball_nba',
  'sport_title': 'NBA',
  'commence_time': '2024-03-06T03:05:57Z',
  'home_team': 'Denver Nuggets',
  'away_team': 'Phoenix Suns',
  'bookmakers': [{'key': 'draftkings',
    'title': 'DraftKings',
    'last_update': '2024-03-06T04:56:38Z',
    'markets': [{'key': 'h2h',
      'last_update': '2024-03-06T04:56:38Z',
      'outcomes': [{'name': 'Denver Nuggets', 'price': 130},
       {'name': 'Phoenix Suns', 'price': -166}]},
     {'key': 'spreads',
      'last_update': '2024-03-06T04:56:38Z',
      'outcomes': [{'name': 'Denver Nuggets', 'price': -120, 'point': 2.5},
       {'name': 'Phoenix Suns', 'price': -110, 'point': -2.5}]}]},
   {'key': 'pointsbetus',
    'title': 'PointsBet (US)',
    'last_update': '2024-03-06T04:56:11Z',
    'markets': [{'key': 'h2h',
      'last_update': '2024-03-06T04:56:11Z',
      'outcomes': [{'name': 'Denver Nuggets', 'price': 220},
       {'name': 'Phoenix Suns', 'price': -290}]

#### Initial Data Ingestion
- Will code most of this in pandas but I need to see if a spark/polars dataframe would work better
long term

In [50]:
import pandas as pd

sportsbook = pd.DataFrame(sportsbook_data)

In [18]:
import polars as pl

pl_sportsbook = pl.DataFrame(sportsbook_data)

In [19]:
# Sample pyspark code

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# JSON data
api_response = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob", "age": 30},
    {"id": 3, "name": "Charlie", "age": 22},
]

# Define the schema for the DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
])

# Create a PySpark DataFrame
df = spark.createDataFrame(api_response, schema=schema)

# Show the DataFrame
df.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 22|
+---+-------+---+



#### Starting to explore the API

In [70]:
json_normalize(json_normalize(sportsbook["markets"].explode())["outcomes"].explode())

,name,price,point
0,Denver Nuggets,370,NaN
1,Phoenix Suns,-560,NaN
2,Denver Nuggets,-115,9.5
3,Phoenix Suns,-115,-9.5
4,Denver Nuggets,350,NaN
5,Phoenix Suns,-500,NaN
6,Denver Nuggets,-120,9.5
7,Phoenix Suns,-105,-9.5
8,Denver Nuggets,290,NaN
9,Phoenix Suns,-400,NaN


In [46]:
from pandas import json_normalize

sportsbook["bookmakers"][1][1]

{'key': 'pointsbetus',
 'title': 'PointsBet (US)',
 'last_update': '2024-03-06T03:47:00Z',
 'markets': [{'key': 'h2h',
   'last_update': '2024-03-06T03:47:00Z',
   'outcomes': [{'name': 'Denver Nuggets', 'price': -400},
    {'name': 'Phoenix Suns', 'price': 300}]},
  {'key': 'spreads',
   'last_update': '2024-03-06T03:47:00Z',
   'outcomes': [{'name': 'Denver Nuggets', 'price': -112, 'point': -8.0},
    {'name': 'Phoenix Suns', 'price': -112, 'point': 8.0}]}]}

In [36]:
response.headers # This is useful to see how many requests I have left for the month

{'Date': 'Wed, 06 Mar 2024 03:47:23 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2763', 'Connection': 'keep-alive', 'X-Requests-Used': '2', 'X-Requests-Remaining': '498', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'Apigw-Requestid': 'UMET0hf_IAMEYug='}

In [103]:
import pandas as pd

# Your nested JSON data
data = {
    'id': 'b2b80d981ff4426f188472bf545a5325',
    'sport_key': 'basketball_nba',
    'sport_title': 'NBA',
    'commence_time': '2024-03-06T03:05:57Z',
    'home_team': 'Denver Nuggets',
    'away_team': 'Phoenix Suns',
    'bookmakers': [
        {
            'key': 'draftkings',
            'title': 'DraftKings',
            'last_update': '2024-03-06T04:56:38Z',
            'markets': [
                {
                    'key': 'h2h',
                    'last_update': '2024-03-06T04:56:38Z',
                    'outcomes': [
                        {'name': 'Denver Nuggets', 'price': 130},
                        {'name': 'Phoenix Suns', 'price': -166}
                    ]
                },
                {
                    'key': 'spreads',
                    'last_update': '2024-03-06T04:56:38Z',
                    'outcomes': [
                        {'name': 'Denver Nuggets', 'price': -120, 'point': 2.5},
                        {'name': 'Phoenix Suns', 'price': -110, 'point': -2.5}
                    ]
                }
            ]
        }
    ]
}

# Flatten the nested structure using json_normalize with prefixes
df_bookmakers = pd.json_normalize(sportsbook_data, 'bookmakers', ['id', 'sport_key', 'sport_title', 'commence_time', 'home_team', 'away_team'], record_prefix='bookmaker_')
df_markets = pd.json_normalize(df_bookmakers.to_dict(orient='records'), 'bookmaker_markets', ['id', 'bookmaker_key', 'bookmaker_title', 'bookmaker_last_update'], record_prefix='market_')
df_outcomes = pd.json_normalize(df_markets.to_dict(orient='records'), 'market_outcomes', ['id', 'bookmaker_key', 'market_key', 'market_last_update'], record_prefix='outcome_')

# Combine the DataFrames on common columns
result_df = pd.merge(df_outcomes, df_markets, how='outer', on=['id', 'bookmaker_key', 'market_key', 'market_last_update'])
result_df = pd.merge(result_df, df_bookmakers, how='outer', on=['id', 'bookmaker_key', 'bookmaker_title', 'bookmaker_last_update'])

# Drop unnecessary columns
result_df = result_df.drop(columns=['bookmaker_markets', 'market_outcomes'])

# # Rename columns if needed
# result_df.columns = ['ID', 'Sport_Key', 'Sport_Title', 'Commence_Time', 'Home_Team', 'Away_Team', 'Bookmaker_Key', 'Bookmaker_Title', 'Last_Update', 'Market_Key', 'Market_Last_Update', 'Outcome_Name', 'Outcome_Price', 'Outcome_Point']

# Display the DataFrame
result_df.shape


(330, 14)

In [106]:
# Need to create a dynamic function that can read these nested results without having to manually enter the keys

#### Preview sample dataset, top 40 results for NBA

In [105]:
result_df.head(40)

,outcome_name,outcome_price,outcome_point,id,bookmaker_key,market_key,market_last_update,bookmaker_title,bookmaker_last_update,sport_key,sport_title,commence_time,home_team,away_team
0,Los Angeles Lakers,-145,NaN,589eed81cc5543b6147d9b6b1c973c08,betmgm,h2h,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
1,Sacramento Kings,120,NaN,589eed81cc5543b6147d9b6b1c973c08,betmgm,h2h,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
2,Los Angeles Lakers,-110,-2.5,589eed81cc5543b6147d9b6b1c973c08,betmgm,spreads,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
3,Sacramento Kings,-110,2.5,589eed81cc5543b6147d9b6b1c973c08,betmgm,spreads,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
4,Los Angeles Lakers,-110,-2.5,589eed81cc5543b6147d9b6b1c973c08,betonlineag,spreads,2024-03-06T04:56:38Z,BetOnline.ag,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
5,Sacramento Kings,-110,2.5,589eed81cc5543b6147d9b6b1c973c08,betonlineag,spreads,2024-03-06T04:56:38Z,BetOnline.ag,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
6,Los Angeles Lakers,-137,NaN,589eed81cc5543b6147d9b6b1c973c08,betrivers,h2h,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
7,Sacramento Kings,116,NaN,589eed81cc5543b6147d9b6b1c973c08,betrivers,h2h,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
8,Los Angeles Lakers,-109,-2.5,589eed81cc5543b6147d9b6b1c973c08,betrivers,spreads,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
9,Sacramento Kings,-112,2.5,589eed81cc5543b6147d9b6b1c973c08,betrivers,spreads,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
